In [1]:
# coding:utf-8
import arrow
import qgrid
import seaborn as sns
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import jieba
    

from sklearn.model_selection import train_test_split


%matplotlib inline

sns.set(style='whitegrid')

In [2]:
trans_prov_allcount = pd.read_csv('../result_data/article_predict_result.csv',encoding='utf-8',index_col = False, delimiter=',' )
trans_prov_allcount.head()

,code,company,label_property,title,url,article
0,HK0001,长和,中性,赫斯基首季盈利增32%胜预期,https://news.10jqka.com.cn/m611120552/,"<p>【财华社讯】<a data-code=""HK0001"" data-marketid=""..."
1,HK0001,长和,中性,李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,<p><span>图品来源：视觉中国</span></p><p>文 | 财联社 万千</p>...
2,HK0001,长和,中性,东兴证券2019年4月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,"<p class=""profile_meta"">\n<label class=""profil..."
3,HK0001,长和,中性,李泽钜接棒一年 长和系继续全球收购,https://news.10jqka.com.cn/m610461495/,<p>时代周报记者蔡颖发自广州</p><p>距“超人”李嘉诚退休已经快一年。在这一年的时间里...
4,HK0001,长和,利好,杰富瑞：若淡马锡售10%屈臣氏持股 长和净资产将升15%,https://news.10jqka.com.cn/m610409446/,"<p>杰富瑞：若淡马锡成功出售10%屈臣氏持股，<a data-code=""HK0001"" ..."


In [3]:
jieba.load_userdict("../corpus/Organization-Names-Corpus.txt")
jieba.load_userdict("../corpus/short-company.txt")
jieba.load_userdict("../corpus/THUOCL_caijing.txt")
jieba.load_userdict("../corpus/company_dict.txt")
jieba.load_userdict("../corpus/username.txt")


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/sz/zfnvjpt16s9_2572h8k9ymd00000gn/T/jieba.cache
Loading model cost 0.843 seconds.
Prefix dict has been built succesfully.


In [4]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
  
def seg_sentence(sentence):  
    outstr = []
    sentence_seged = jieba.cut(sentence.strip())  
    stopwords = stopwordslist('../corpus/stopword.txt')  # 这里加载停用词的路径  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr.append(word)   
    return outstr 
def clearstr(line_str):
    rule = re.compile(r"[^\u4e00-\u9fa5]")
    line_str = rule.sub('',line_str)
    rule = re.compile(u"[^\u4e00-\u9fa5]")
    line_str = rule.sub('',line_str)
    return re.sub("[\d+\s+\.\!\/_,$%^*(+\"\'<>=【】\-]+|[+——！，。？、~@#￥%……&*（）：]+", "",line_str)



In [5]:
import re
trans_prov_allcount['title'] =trans_prov_allcount['title'].apply(clearstr)
trans_prov_allcount['article'] =trans_prov_allcount['article'].apply(clearstr)
trans_prov_allcount.head()

,code,company,label_property,title,url,article
0,HK0001,长和,中性,赫斯基首季盈利增胜预期,https://news.10jqka.com.cn/m611120552/,财华社讯长和旗下加拿大赫斯基能源公布首季业绩盈利亿元加元下同同比上升扣除特殊项目后每股盈利元...
1,HK0001,长和,中性,李泽钜接手后的屈臣氏前有丝芙兰后有腾讯阿里争买股份谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,图品来源视觉中国文财联社万千去年月李嘉诚退休李泽钜正式接管父亲的万亿商业帝国同时接手了旗下已...
2,HK0001,长和,中性,东兴证券年月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,功能介绍推送东兴研究所的研究成果来服务机构投资者外资风险偏好扰动放大市场情绪波动随着减税等一...
3,HK0001,长和,中性,李泽钜接棒一年长和系继续全球收购,https://news.10jqka.com.cn/m610461495/,时代周报记者蔡颖发自广州距超人李嘉诚退休已经快一年在这一年的时间里外界好奇新掌门人李泽钜秉承...
4,HK0001,长和,利好,杰富瑞若淡马锡售屈臣氏持股长和净资产将升,https://news.10jqka.com.cn/m610409446/,杰富瑞若淡马锡成功出售屈臣氏持股长和的净资产值将可提升


In [7]:
trans_prov_allcount['article_words'] =trans_prov_allcount['article'].apply(seg_sentence)
trans_prov_allcount.head()

,code,company,label_property,title,url,article,article_words
0,HK0001,长和,中性,赫斯基首季盈利增胜预期,https://news.10jqka.com.cn/m611120552/,财华社讯长和旗下加拿大赫斯基能源公布首季业绩盈利亿元加元下同同比上升扣除特殊项目后每股盈利元...,"[财华社, 讯, 长和, 旗下, 加拿大, 赫斯基, 能源, 公布, 首季, 业绩, 盈利,..."
1,HK0001,长和,中性,李泽钜接手后的屈臣氏前有丝芙兰后有腾讯阿里争买股份谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,图品来源视觉中国文财联社万千去年月李嘉诚退休李泽钜正式接管父亲的万亿商业帝国同时接手了旗下已...,"[图品, 来源, 视觉, 中国, 文, 财联社, 万千, 去年, 月, 李嘉诚, 退休, 李..."
2,HK0001,长和,中性,东兴证券年月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,功能介绍推送东兴研究所的研究成果来服务机构投资者外资风险偏好扰动放大市场情绪波动随着减税等一...,"[功能, 介绍, 推送, 东兴, 研究所, 研究成果, 服务, 机构, 投资者, 外资, 风..."
3,HK0001,长和,中性,李泽钜接棒一年长和系继续全球收购,https://news.10jqka.com.cn/m610461495/,时代周报记者蔡颖发自广州距超人李嘉诚退休已经快一年在这一年的时间里外界好奇新掌门人李泽钜秉承...,"[时代, 周报, 记者, 蔡颖, 发自, 广州, 超人, 李嘉诚, 退休, 已经, 快, 一..."
4,HK0001,长和,利好,杰富瑞若淡马锡售屈臣氏持股长和净资产将升,https://news.10jqka.com.cn/m610409446/,杰富瑞若淡马锡成功出售屈臣氏持股长和的净资产值将可提升,"[杰富瑞, 淡马锡, 成功, 出售, 屈臣氏, 持股, 长和, 净资产值, 提升]"


In [13]:
combined = trans_prov_allcount['article_words'].values

In [14]:
def preprocess_toint(x):
    cate_dic = {'利好':1, '中性':0, '利空':-1}
    cate = x
    result_str = cate_dic[cate]
    return result_str
trans_prov_allcount['label_int'] = trans_prov_allcount['label_property'].apply(preprocess_toint)
trans_prov_allcount.head()

,code,company,label_property,title,url,article,article_words,label_int
0,HK0001,长和,中性,赫斯基首季盈利增胜预期,https://news.10jqka.com.cn/m611120552/,财华社讯长和旗下加拿大赫斯基能源公布首季业绩盈利亿元加元下同同比上升扣除特殊项目后每股盈利元...,"[财华社, 讯, 长和, 旗下, 加拿大, 赫斯基, 能源, 公布, 首季, 业绩, 盈利,...",0
1,HK0001,长和,中性,李泽钜接手后的屈臣氏前有丝芙兰后有腾讯阿里争买股份谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,图品来源视觉中国文财联社万千去年月李嘉诚退休李泽钜正式接管父亲的万亿商业帝国同时接手了旗下已...,"[图品, 来源, 视觉, 中国, 文, 财联社, 万千, 去年, 月, 李嘉诚, 退休, 李...",0
2,HK0001,长和,中性,东兴证券年月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,功能介绍推送东兴研究所的研究成果来服务机构投资者外资风险偏好扰动放大市场情绪波动随着减税等一...,"[功能, 介绍, 推送, 东兴, 研究所, 研究成果, 服务, 机构, 投资者, 外资, 风...",0
3,HK0001,长和,中性,李泽钜接棒一年长和系继续全球收购,https://news.10jqka.com.cn/m610461495/,时代周报记者蔡颖发自广州距超人李嘉诚退休已经快一年在这一年的时间里外界好奇新掌门人李泽钜秉承...,"[时代, 周报, 记者, 蔡颖, 发自, 广州, 超人, 李嘉诚, 退休, 已经, 快, 一...",0
4,HK0001,长和,利好,杰富瑞若淡马锡售屈臣氏持股长和净资产将升,https://news.10jqka.com.cn/m610409446/,杰富瑞若淡马锡成功出售屈臣氏持股长和的净资产值将可提升,"[杰富瑞, 淡马锡, 成功, 出售, 屈臣氏, 持股, 长和, 净资产值, 提升]",1


In [19]:
y=trans_prov_allcount['label_int'].values

In [20]:
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence
import multiprocessing

cpu_count = multiprocessing.cpu_count() # 4
vocab_dim = 100
n_iterations = 10  # ideally more..
n_exposures = 10 # 所有频数超过10的词语
window_size = 7
n_epoch = 4
input_length = 100
maxlen = 100

def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引,(k->v)=>(v->k)
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量, (word->model(word))

        def parse_dataset(combined): # 闭包-->临时使用
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0) # freqxiao10->0
                data.append(new_txt)
            return data # word=>index
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print('No data provided...')


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined) # input: list
    model.train(combined, total_examples=model.corpus_count, epochs=model.iter)
    model.save('../model/Word2vec_model.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined

print('Training a Word2vec model...')
index_dict, word_vectors,combined=word2vec_train(combined)

Using TensorFlow backend.


Training a Word2vec model...


/Users/zhuangzhuanghuang/miniconda2/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:60: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
/Users/zhuangzhuanghuang/miniconda2/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [21]:
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
np.random.seed(1337)  # For Reproducibility
import sys
sys.setrecursionlimit(1000000)
import yaml
import keras

batch_size = 32


def get_data(index_dict,word_vectors,combined,y):

    n_symbols = len(index_dict) + 1  # 所有单词的索引数，频数小于10的词语索引为0，所以加1
    embedding_weights = np.zeros((n_symbols, vocab_dim)) # 初始化 索引为0的词语，词向量全为0
    for word, index in index_dict.items(): # 从索引为1的词语开始，对每个词语对应其词向量
        embedding_weights[index, :] = word_vectors[word]
    x_train, x_test, y_train, y_test = train_test_split(combined, y, test_size=0.2)
    y_train = keras.utils.to_categorical(y_train,num_classes=3) 
    y_test = keras.utils.to_categorical(y_test,num_classes=3)
    # print x_train.shape,y_train.shape
    return n_symbols,embedding_weights,x_train,y_train,x_test,y_test


##定义网络结构
def train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test):
    print('Defining a Simple Keras Model...')
    model = Sequential()  # or Graph or whatever
    model.add(Embedding(output_dim=vocab_dim,
                        input_dim=n_symbols,
                        mask_zero=True,
                        weights=[embedding_weights],
                        input_length=input_length))  # Adding Input Length
    model.add(LSTM(output_dim=50, activation='tanh', inner_activation='hard_sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax')) # Dense=>全连接层,输出维度=1
    model.add(Activation('softmax'))

    #print 'Compiling the Model...'
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])

    #print "Train..." # batch_size=32
    model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch,verbose=1)

    #print "Evaluate..."
    score = model.evaluate(x_test, y_test,
                                batch_size=batch_size)

    yaml_string = model.to_yaml()
    with open('../model/lstm.yml', 'w') as outfile:
        outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
    model.save_weights('../model/lstm.h5')
    print('Test score:', score)

#print 'Setting up Arrays for Keras Embedding Layer...'
n_symbols,embedding_weights,x_train,y_train,x_test,y_test=get_data(index_dict, word_vectors,combined,y)
print("x_train.shape and y_train.shape:")
print(x_train.shape,y_train.shape)
train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test)

/Users/zhuangzhuanghuang/miniconda2/envs/3point6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/zhuangzhuanghuang/miniconda2/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", units=50, recurrent_activation="hard_sigmoid")`


x_train.shape and y_train.shape:
(29736, 100) (29736, 3)
Defining a Simple Keras Model...
Epoch 1/4
29736/29736 [==============================] - 99s 3ms/step - loss: 0.9551 - acc: 0.5856
Epoch 2/4
29736/29736 [==============================] - 101s 3ms/step - loss: 0.9129 - acc: 0.6288
Epoch 3/4
29736/29736 [==============================] - 99s 3ms/step - loss: 0.8916 - acc: 0.6499
Epoch 4/4
7435/7435 [==============================] - 5s 703us/step
Test score: [0.8690863993557492, 0.6711499663672355]


In [23]:
"""
预测
"""
import jieba
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence

import yaml
from keras.models import model_from_yaml
np.random.seed(1337)  # For Reproducibility
import sys
sys.setrecursionlimit(1000000)

# define parameters
maxlen = 100

def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引,(k->v)=>(v->k)
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量, (word->model(word))

        def parse_dataset(combined): # 闭包-->临时使用
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0) # freqxiao10->0
                data.append(new_txt)
            return data # word=>index
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print('No data provided...')


def input_transform(str_cln):
    str_cln= clearstr(str_cln)
    str_arr = seg_sentence(str_cln)
    #words=jieba.lcut(string)
    words=np.array(str_arr).reshape(1,-1)
    model=Word2Vec.load('../model/Word2vec_model.pkl')
    _,_,combined=create_dictionaries(model,words)
    return combined


def lstm_predict(string):
    print('loading model......')
    with open('../model/lstm.yml', 'r') as f:
        yaml_string = yaml.load(f)
    model = model_from_yaml(yaml_string)

    print('loading weights......')
    model.load_weights('../model/lstm.h5')
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    data=input_transform(string)
    data.reshape(1,-1)
    #print data
    result=model.predict_classes(data)
    print(result)
    if result[0]==1:
        print(string,'利好')
    elif result[0]==0:
        print(string,' 中性')
    else:
        print(string,' 利空')

In [26]:
string = "智通财经APP讯，煜新控股(01048)发布公告，该公司截至2018年10月31日止6个月，实现收益559.6万美元，去年同期收益为0;股东应占期间亏损726.9万美元，同比增加108.16%;每股亏损3.8美仙。公告显示，镀锡钢片制造业务于截至2015年4月30日止财政年度暂停营运，经历活化过程后于2018年5月恢复营运，并于该期间产生销售收益约5600万美元。毛损乃由于镀锡钢片制造业务当时处于复产早期，厂房及机械仍处于测试阶段，销量能达致阶段性收支平衡"

lstm_predict(string)

loading model......
loading weights......


/Users/zhuangzhuanghuang/miniconda2/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[2]
智通财经APP讯，煜新控股(01048)发布公告，该公司截至2018年10月31日止6个月，实现收益559.6万美元，去年同期收益为0;股东应占期间亏损726.9万美元，同比增加108.16%;每股亏损3.8美仙。公告显示，镀锡钢片制造业务于截至2015年4月30日止财政年度暂停营运，经历活化过程后于2018年5月恢复营运，并于该期间产生销售收益约5600万美元。毛损乃由于镀锡钢片制造业务当时处于复产早期，厂房及机械仍处于测试阶段，销量能达致阶段性收支平衡  利空
